# 股票预测模型工作流

---
### 工作流说明
1.  **阶段零 (Setup)**: 导入库、加载配置。
2.  **阶段一 (Data Pipeline)**: 独立运行。负责处理并保存数据，生成 L2 特征数据缓存。
3.  **阶段二 (Model Pipeline)**: 独立运行。包含三个子步骤：
    - **2.1 HPO**: 自动调参。
    - **2.2 (预处理)**: 智能地加载或生成 L3 预处理数据缓存
    - **2.3 (模型训练)**: 使用 L3 缓存进行高效的模型训练。
    - **2.4 (评估)**: 对训练结果进行聚合与可视化。

## 0. 通用设置与导入

In [1]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# 设置 Matplotlib 样式 (可以在 main_train 外部设置)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# --- 动态导入 main_train ---
project_root = str(Path().resolve())
if project_root not in sys.path:
    sys.path.append(project_root)

# 从我们统一的核心引擎中导入所有需要的函数和模块
from main_train import (
    run_load_config_and_modules,
    run_all_data_pipeline,
    run_preprocess_l3_cache,
    run_hpo_train,
    run_all_models_train,
    run_performance_evaluation,
    run_results_visualization
)

# --- 加载全局配置和模块 ---
# 在 Notebook 的生命周期中，我们只加载一次
config, modules = run_load_config_and_modules()

if not (config and modules):
    raise RuntimeError("环境初始化失败，请检查配置文件路径和模块导入。")

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\main_train.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


--- 正在初始化环境：加载配置与模块... ---
INFO: 底层并行计算库线程数已设置为: 4
INFO: 项目模块导入成功。
SUCCESS: 配置已从 'configs/config.yaml' 加载。


# **阶段一：数据准备与特征工程**

In [2]:
USE_LATEST_DATE = True
if config and modules:
    print(f"--- INFO: 日期模式 -> {'使用最新日期' if USE_LATEST_DATE else '使用配置文件中的固定日期'} ---")
    
    # 将开关状态传递给核心引擎函数
    run_all_data_pipeline(config, modules, use_today_as_end_date=USE_LATEST_DATE)

--- INFO: 日期模式 -> 使用最新日期 ---
=== 阶段一：数据准备与特征工程 ===
INFO: 已启用动态日期模式，将使用今天的日期 '2025-10-27' 作为 end_date。
INFO: 日期已解析. Start Date: 2010-10-27, End Date: 2025-10-27
INFO: 正在尝试登录 Baostock...
login success!
INFO: Baostock API 登录成功。
INFO: 未在配置中提供有效的 Tushare Token，将跳过 Tushare 相关数据。
开始执行数据管道协调任务...

需要为以下 7 只股票生成新数据: ['TCL科技 (000100.SZ)', '兴业矿业 (000426.SZ)', '孚日股份 (002083.SZ)', '宜华健康 (000150.SZ)', '新宁物流 (300013.SZ)', '佳云科技 (300242.SZ)', '精功科技 (002006.SZ)']

--- 正在准备所有全局市场数据 (此过程只运行一次) ---
--- 开始生成市场广度数据 ---
  - 正在获取指数成分股列表...


下载成分股日线数据:   0%|          | 0/300 [00:00<?, ?it/s]

  - 正在从本地缓存加载 sh.600000 (sh.600000) 的原始日线数据...
  - 正在从本地缓存加载 sh.600009 (sh.600009) 的原始日线数据...
  - 正在从本地缓存加载 sh.600010 (sh.600010) 的原始日线数据...
  - 正在从本地缓存加载 sh.600011 (sh.600011) 的原始日线数据...
  - 正在从本地缓存加载 sh.600015 (sh.600015) 的原始日线数据...
  - 正在从本地缓存加载 sh.600016 (sh.600016) 的原始日线数据...
  - 正在从本地缓存加载 sh.600018 (sh.600018) 的原始日线数据...
  - 正在从本地缓存加载 sh.600019 (sh.600019) 的原始日线数据...
  - 正在从本地缓存加载 sh.600023 (sh.600023) 的原始日线数据...
  - 正在从本地缓存加载 sh.600025 (sh.600025) 的原始日线数据...
  - 正在从本地缓存加载 sh.600026 (sh.600026) 的原始日线数据...
  - 正在从本地缓存加载 sh.600027 (sh.600027) 的原始日线数据...
  - 正在从本地缓存加载 sh.600028 (sh.600028) 的原始日线数据...
  - 正在从本地缓存加载 sh.600029 (sh.600029) 的原始日线数据...
  - 正在从本地缓存加载 sh.600030 (sh.600030) 的原始日线数据...
  - 正在从本地缓存加载 sh.600031 (sh.600031) 的原始日线数据...
  - 正在从本地缓存加载 sh.600036 (sh.600036) 的原始日线数据...
  - 正在从本地缓存加载 sh.600039 (sh.600039) 的原始日线数据...
  - 正在从本地缓存加载 sh.600048 (sh.600048) 的原始日线数据...
  - 正在从本地缓存加载 sh.600050 (sh.600050) 的原始日线数据...
  - 正在从本地缓存加载 sh.600061 (sh.600061) 的原始日线数据...
  - 正在从本地缓存加载

d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:107: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_df = closes_df.pct_change().fillna(0)


--- 市场广度数据已生成并缓存至 data\data_cache\market_breadth_2010-10-27_2025-09-30.pkl ---
INFO: 正在下载 4 个外部市场的数据: ['SPY', 'QQQ', 'GLD', 'TLT']
  - 正在为 SPY 获取美股数据...
  - 正在从 Yahoo Finance 下载 SPY 的数据...


...

  - 警告 [YF]: 下载 TLT 数据失败 (尝试 1/3). 将在 0.50 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:310: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['TLT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 TLT 数据失败 (尝试 2/3). 将在 1.00 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.


d:\Project\Command\Python\Neural\Wolf_of_Wall_Street\data_process\get_data.py:310: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date_inclusive, progress=False)

1 Failed download:
['TLT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


  - 警告 [YF]: 下载 TLT 数据失败 (尝试 3/3). 将在 2.00 秒后重试. 错误: yfinance returned an empty DataFrame for TLT.
  - 错误 [YF]: 下载 TLT 数据在 3 次尝试后仍失败。
--- 正在获取市场情绪数据 (恐慌指数) ---
  - 错误 [akshare]: 获取市场情绪数据失败: module 'akshare' has no attribute 'idx_ivix'
--- 所有全局市场数据准备完毕 ---


--- 正在为 TCL科技 (000100.SZ) 生成【训练用】历史特征 ---
  - 数据窗口: 2010-10-27 to 2025-09-30
  - 正在从本地缓存加载 TCL科技 (sz.000100) 的原始日线数据...
  - INFO: 正在为 'TCL科技的行业指数' (159928.SZ) 获取指数数据...
  - 正在从 Baostock 下载 TCL科技的行业指数 (159928.SZ) 的日线行情...
  - WARNING [BS]: 未能获取到 TCL科技的行业指数 (159928.SZ) 在指定日期范围的数据。
    - 警告: 无法直接获取指数 'TCL科技的行业指数' (159928.SZ)。将尝试在本地合成等权重指数...
    - 错误: 在 API 映射表中未找到指数 'TCL科技的行业指数' (159928.SZ) 的成分股查询接口。
  - INFO: 正在为 '基准指数' (sh.000300) 获取指数数据...
  - 正在从 Baostock 下载 基准指数 (sh.000300) 的日线行情...
  - INFO: 已将 基准指数 (sh.000300) 的数据缓存至 data\data_cache\raw_ohlcv\raw_sh_000300_2010-10-27_2025-09-30.pkl
    - SUCCESS: 已直接从 Baostock 获取到 '基准指数' (sh.000300) 的数据。
INFO: 开始特征计算流水线...
  - [Calculating Features] Running: Technical Indicators...
    - Calcul

# **阶段二：模型训练与评估**

### 2.1 数据预加载与全局预处理 (L3 缓存)

In [3]:
if config and modules:
    # 调用核心引擎中的预处理函数
    # force_reprocess=True/False 可以方便地控制是否重建缓存
    global_data_cache = run_preprocess_l3_cache(config, modules, force_reprocess=False)

=== 工作流阶段 2.1：为模型预处理数据 (L3 缓存) ===
INFO: 开始执行预处理流程 (分块保存模式)...



正在预处理股票:   0%|          | 0/7 [00:00<?, ?it/s]

--- INFO: L3 缓存文件已在磁盘上准备就绪。 ---
--- 阶段 2.1 成功完成。 ---


### 2.2 超参数优化

In [4]:
# 同样，可以通过一个简单的开关来控制是否运行
RUN_HPO = False

if RUN_HPO and config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的 HPO 函数
    # HPO 的结果会自动更新到内存中的 config 字典里
    run_hpo_train(config, modules, global_data_cache)

### 2.3 模型训练

In [ ]:
if config and modules and 'global_data_cache' in locals():
    # 调用核心引擎中的模型训练函数
    # 同样可以通过开关控制是否强制重训
    all_ic_history = run_all_models_train(
        config, 
        modules,  
        force_retrain_base=False, 
        force_retrain_fuser=True
    )

=== 工作流阶段 2.3：训练所有模型 ===


处理股票模型:   0%|          | 0/7 [00:00<?, ?it/s]


--- 2.3.1 为 TCL科技 (000100.SZ) 训练基础模型 ---
--- 开始为 TCL科技 (000100.SZ) 进行 LGBM 模型训练 ---
INFO: 未检测到任何历史记录，将从头开始全新训练。
INFO: 开始对 TCL科技 (000100.SZ) 进行跨 62 folds 的前向验证...


正在 TCL科技 上训练 LGBM:   0%|          | 0/62 [00:00<?, ?it/s]

INFO: 滚动训练成功完成，准备训练最终模型。
INFO: 正在训练最终模型...
SUCCESS: 新版本 (20250930) 模型已保存: lgbm_model_20250930.pkl
INFO: 整个训练流程（包括最终模型）成功完成，已移除进度文件。
--- 开始为 TCL科技 (000100.SZ) 进行 LSTM 模型训练 ---
INFO: 未检测到任何历史记录，将从头开始全新训练。
INFO: PyTorch LSTMBuilder initialized with device: CUDA
INFO: DataLoader will use 8 parallel workers.
INFO: Automatic Mixed Precision (AMP) is ENABLED (float16).
INFO: 开始对 TCL科技 (000100.SZ) 进行跨 62 folds 的前向验证...


...

    - Epochs (LSTM):   0%|          | 0/75 [00:00<?, ?it/s]

    - INFO: 早停机制已在第 68 轮触发。
    - Fold finished. Best validation loss: 0.002776 at epoch 18


    - Epochs (LSTM):   0%|          | 0/75 [00:00<?, ?it/s]

    - INFO: 早停机制已在第 51 轮触发。
    - Fold finished. Best validation loss: 0.001782 at epoch 1
INFO: 滚动训练成功完成，准备训练最终模型。
INFO: 正在训练最终模型...
    - INFO: Starting final LSTM model training on all data...
    - INFO: Training final model for a fixed 50 epochs.


    - Final Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

### 2.4 结果聚合、评估与可视化

In [ ]:
if config and modules and 'all_ic_history' in locals() and all_ic_history:
    # 调用核心引擎中的评估和可视化函数
    evaluation_summary, backtest_summary, final_eval_df = run_performance_evaluation(config, modules, all_ic_history)
    
    # 只有在有结果时才进行可视化
    if evaluation_summary is not None or backtest_summary is not None:
        run_results_visualization(config, modules, evaluation_summary, backtest_summary, final_eval_df)
    else: print('无可视化结果')